# Compare Lipinski‐rule Distributions
### Hotelling T², MMD, and Binary Rule‑Signature Tests

1. **Install** dependencies.
2. **Upload** two CSVs with `MolWt`, `LogP`, `HBD`, `HBA`.
3. **Adjust** parameters if needed.
4. **Run** the analysis cell.

In [1]:
!pip install -q pandas numpy scipy scikit-learn pingouin hyppo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 50.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.


In [10]:
import itertools, numpy as np, pandas as pd
from scipy.stats import chi2_contingency
from scipy.spatial.distance import jensenshannon
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from hyppo.ksample import MMD

# -------------------------------------------
# I/O helpers
# -------------------------------------------
def load_lipo_csv(path):
    '''
    Load CSV and return a NumPy array (n,4) for the Lipinski columns.
    '''
    df = pd.read_csv(path)
    cols = ['MolWt', 'LogP', 'HBD', 'HBA']
    missing = set(cols) - set(df.columns)
    if missing:
        raise ValueError(f"{path} missing columns: {missing}")
    return df[cols].dropna().astype(float).values

def binary_patterns(df):
    '''
    Return counts of the 16 binary Lipinski rule‑signature patterns.
    '''
    sig = pd.DataFrame({
        'MW_ok'  : (df['MolWt'] <= 500).astype(int),
        'LogP_ok': (df['LogP']  <=   5).astype(int),
        'HBD_ok' : (df['HBD']   <=   5).astype(int),
        'HBA_ok' : (df['HBA']   <=  10).astype(int)
    })
    patterns = sig.agg(lambda r: ''.join(r.astype(str)), axis=1)
    all_codes = [''.join(p) for p in itertools.product('01', repeat=4)]
    return patterns.value_counts().reindex(all_codes, fill_value=0).astype(int)

# -------------------------------------------
# Main comparison function
# -------------------------------------------
def compare_distributions(path_a, path_b, labels=('A', 'B'),
                          sample_size=1000, n_perm=300, n_workers=-1,
                          random_seed=42):
    '''
    Compare two Lipinski datasets with Hotelling T², MMD, and binary‑signature tests.
    '''
    rng = np.random.RandomState(random_seed)

    X = load_lipo_csv(path_a)
    Y = load_lipo_csv(path_b)

    if sample_size is not None and sample_size < min(len(X), len(Y)):
      idx_x = rng.choice(len(X), size=sample_size, replace=False)
      idx_y = rng.choice(len(Y), size=sample_size, replace=False)
      X = X[idx_x]
      Y = Y[idx_y]


    # Hotelling T²
    try:
        import pingouin as pg
        res = pg.multivariate_ttest(X, Y)
        t2, p_hot = res['T2'].values[0], res['pval'].values[0]
        print(f"\\nHotelling T² (4‑D means): T² = {t2:.3f}  |  p = {p_hot:.3e}")
    except ImportError:
        print("\\nInstall 'pingouin' for Hotelling T²:  pip install pingouin")

    # MMD
    # --- Standardize X and Y together ------------------------------------
    Z = np.vstack([X, Y])
    scaler = StandardScaler()
    Z_scaled = scaler.fit_transform(Z)
    X_scaled = Z_scaled[:len(X)]
    Y_scaled = Z_scaled[len(X):]

    dists = pairwise_distances(Z)
    median_dist = np.median(dists)
    gamma = 1.0 / (2 * median_dist**2)

    mmd = MMD(compute_kernel="gaussian", gamma=gamma, bias=False)
    stat, p_mmd = mmd.test(X, Y, reps=n_perm, workers=n_workers)

    print("\\nMMD (Maximum Mean Discrepancy) test:")
    print(f"  MMD statistic        : {stat:.4f}")
    print(f"  Permutation p‑value  : {p_mmd:.3e}")

    # Binary rule‑signature
    cnt_a = binary_patterns(pd.DataFrame(X, columns=['MolWt','LogP','HBD','HBA']))
    cnt_b = binary_patterns(pd.DataFrame(Y, columns=['MolWt','LogP','HBD','HBA']))

    table = np.vstack([cnt_a, cnt_b])
    mask = table.sum(axis=0) > 0
    table_clean = table[:, mask]

    if table_clean.shape[1] < 2:
        print("Not enough non‑empty rule patterns for a χ² test.")
    else:
        chi2, p_chi, dof, _ = chi2_contingency(table_clean, lambda_=None)
        probs_a = table_clean[0] / table_clean[0].sum()
        probs_b = table_clean[1] / table_clean[1].sum()


        # Chi-squared standardized residuals
        row_totals = table_clean.sum(axis=1, keepdims=True)
        col_totals = table_clean.sum(axis=0, keepdims=True)
        grand_total = table_clean.sum()
        expected = (row_totals @ col_totals) / grand_total
        residuals = (table_clean - expected) / np.sqrt(expected)

        jsd = jensenshannon(probs_a, probs_b, base=2.0)

        print("\\nBinary rule‑signature tests (after removing all‑zero patterns):")
        print(f"  χ² p‑value                  : {p_chi:.3e}")
        print(f"  χ² statistic (dof={dof})     : {chi2:.2f}")
        for i, label in enumerate(labels):
            print(f"  Residuals for {label}:")
            for pat, r in zip(cnt_a.index[mask], residuals[i]):
                print(f"    {pat}: {r:.2f}")
        print(f"  Jensen–Shannon divergence    : {jsd:.3f}")

    diffs = (cnt_a - cnt_b).abs().sort_values(ascending=False)
    print("\\nTop differing 4‑bit patterns (Δ counts):")
    for pat, d in diffs.items():
        print(f"    {pat}: {labels[0]}={cnt_a[pat]}, {labels[1]}={cnt_b[pat]}  (Δ={d})")

In [4]:
from google.colab import files

print("Please upload the FIRST CSV (dataset A)")
uploaded_a = files.upload()
if len(uploaded_a) != 1:
    raise RuntimeError("Upload exactly ONE file for dataset A.")
path_a = next(iter(uploaded_a.keys()))

# --- Upload dataset B -------------------------------------------------
print("Please upload the SECOND CSV (dataset B)")
uploaded_b = files.upload()
if len(uploaded_b) != 1:
    raise RuntimeError("Upload exactly ONE file for dataset B.")
path_b = next(iter(uploaded_b.keys()))

print(f"Using files:\n  A: {path_a}\n  B: {path_b}")

Please upload the FIRST CSV (dataset A)


Saving lipinski_properties_all.csv to lipinski_properties_all.csv
Please upload the SECOND CSV (dataset B)


Saving lipinski_properties_lipinski.csv to lipinski_properties_lipinski.csv
Using files:
  A: lipinski_properties_all.csv
  B: lipinski_properties_lipinski.csv


In [5]:
PARAMS = {
    'sample_size': 5000,
    'n_perm': 1000,
    'n_workers': -1,
    'labels': ('FULL', 'RELAX'),
}

In [11]:
compare_distributions(path_a, path_b,
                      labels=PARAMS['labels'],
                      sample_size=PARAMS['sample_size'],
                      n_perm=PARAMS['n_perm'],
                      n_workers=PARAMS['n_workers'])

\nHotelling T² (4‑D means): T² = 753.489  |  p = 7.032e-156
\nMMD (Maximum Mean Discrepancy) test:
  MMD statistic        : 0.0014
  Permutation p‑value  : 2.191e-26
\nBinary rule‑signature tests (after removing all‑zero patterns):
  χ² p‑value                  : 3.153e-214
  χ² statistic (dof=8)     : 1017.03
  Residuals for FULL:
    0011: -2.31
    0100: 1.22
    0110: -1.00
    0111: -13.34
    1011: -15.89
    1100: 0.71
    1101: -3.67
    1110: -2.92
    1111: 6.90
  Residuals for RELAX:
    0011: 2.31
    0100: -1.22
    0110: 1.00
    0111: 13.34
    1011: 15.89
    1100: -0.71
    1101: 3.67
    1110: 2.92
    1111: -6.90
  Jensen–Shannon divergence    : 0.311
\nTop differing 4‑bit patterns (Δ counts):
    1111: FULL=4987, RELAX=4059  (Δ=928)
    1011: FULL=4, RELAX=517  (Δ=513)
    0111: FULL=0, RELAX=356  (Δ=356)
    1101: FULL=0, RELAX=27  (Δ=27)
    1110: FULL=0, RELAX=17  (Δ=17)
    0011: FULL=5, RELAX=22  (Δ=17)
    0100: FULL=3, RELAX=0  (Δ=3)
    0110: FULL=0, RELAX=2